In [1]:
pip install textblob


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install wordcloud


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore')

In [4]:
trump = pd.read_csv(r"C:\Users\mayur\Downloads\archive (9)\data\data\tweets\alvarodias__election_tweets.csv")
print(trump.head(3))

Empty DataFrame
Columns: [id;created_at;text;retweet_count;favorite_count]
Index: []


In [5]:
print(trump.columns)

Index(['id;created_at;text;retweet_count;favorite_count'], dtype='object')


In [6]:
biden = pd.read_csv(r"C:\Users\mayur\Downloads\archive (8)\247500_totaloutput_9parts.csv", lineterminator='\n')
print(biden.head(2))

          username                              accDesc location  following  \
0  PrimalPolitical  Post-Liberal, Pro-Brexit, Primitive      NaN     1060.0   
1   lukejohnpearce                                  NaN      NaN       27.0   

   followers totaltweets          usercreated         tweetcreated  \
0     7332.0      7483.0  2020-01-03 14:26:06  2020-11-04 10:36:54   
1       62.0        40.0  2017-07-12 03:35:23  2020-11-04 10:36:53   

   retweetcount favoritecount  \
0           0.0           0.0   
1           0.0           0.0   

                                                text          tweetsource  \
0  Righ now I'd say Trump is better positioned, j...  Twitter for Android   
1  trump after pretending he cares about the work...   Twitter for iPhone   

                                           hashtags  
0                                                []  
1  [{'text': 'Elections2020', 'indices': [67, 81]}]  


In [7]:
print(trump.shape)
print(biden.shape)

(0, 1)
(1822076, 13)


In [8]:
trump.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 1 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   id;created_at;text;retweet_count;favorite_count  0 non-null      object
dtypes: object(1)
memory usage: 132.0+ bytes


In [9]:
biden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822076 entries, 0 to 1822075
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   username       object 
 1   accDesc        object 
 2   location       object 
 3   following      float64
 4   followers      float64
 5   totaltweets    object 
 6   usercreated    object 
 7   tweetcreated   object 
 8   retweetcount   float64
 9   favoritecount  object 
 10  text           object 
 11  tweetsource    object 
 12  hashtags       object 
dtypes: float64(3), object(10)
memory usage: 180.7+ MB


In [10]:
# creating a new column 'candidate' todifferentiate
# between tweets of Trump and Biden upon concatination
trump['candidate'] = 'trump'

# biden dataframe
biden['candidate'] = 'biden'

# combining the dataframes
data = pd.concat([trump, biden])

# FInal data shape
print('Final Data Shape :', data.shape)

# View the first 2 rows
print("\nFirst 2 rows:")
print(data.head(3))

Final Data Shape : (1822076, 15)

First 2 rows:
  id;created_at;text;retweet_count;favorite_count candidate         username  \
0                                             NaN     biden  PrimalPolitical   
1                                             NaN     biden   lukejohnpearce   
2                                             NaN     biden     TheTylerWest   

                               accDesc location  following  followers  \
0  Post-Liberal, Pro-Brexit, Primitive      NaN     1060.0     7332.0   
1                                  NaN      NaN       27.0       62.0   
2                      @Twitch Partner      NaN     1020.0     2786.0   

  totaltweets          usercreated         tweetcreated  retweetcount  \
0      7483.0  2020-01-03 14:26:06  2020-11-04 10:36:54           0.0   
1        40.0  2017-07-12 03:35:23  2020-11-04 10:36:53           0.0   
2     21118.0  2011-04-03 15:58:18  2020-11-04 10:36:53           0.0   

  favoritecount                              

In [11]:
data.dropna(inplace=True)

In [12]:
if 'country' in data.columns:
    print(data['country'].value_counts())
else:
    print("Column 'country' not found.")
    print("Available columns:", list(data.columns))


Column 'country' not found.
Available columns: ['id;created_at;text;retweet_count;favorite_count', 'candidate', 'username', 'accDesc', 'location', 'following', 'followers', 'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount', 'favoritecount', 'text', 'tweetsource', 'hashtags']


In [13]:
data.columns = data.columns.str.strip()

if 'country' in data.columns:
    data['country'] = data['country'].replace({
        'United States of America': 'US',
        'United States': 'US'
    })
else:
    print("❌ 'country' column not found")
    print("Available columns:", list(data.columns))


❌ 'country' column not found
Available columns: ['id;created_at;text;retweet_count;favorite_count', 'candidate', 'username', 'accDesc', 'location', 'following', 'followers', 'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount', 'favoritecount', 'text', 'tweetsource', 'hashtags']


In [14]:
print(data.columns)

Index(['id;created_at;text;retweet_count;favorite_count', 'candidate',
       'username', 'accDesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount',
       'favoritecount', 'text', 'tweetsource', 'hashtags'],
      dtype='object')


In [15]:
import plotly.express as px

# Clean column names
data.columns = data.columns.str.strip()

# Check columns
print(data.columns)

# Use correct tweet column name (example: 'text')
tweets_count = (
    data.groupby('candidate')['text']
    .count()
    .reset_index(name='tweet_count')
)

fig = px.bar(
    tweets_count,
    x='candidate',
    y='tweet_count',
    color='candidate',
    color_discrete_map={'Trump': 'pink', 'Biden': 'blue'},
    labels={
        'candidate': 'Candidates',
        'tweet_count': 'Number of Tweets'
    },
    title='Tweets for Candidates'
)

fig.show()


Index(['id;created_at;text;retweet_count;favorite_count', 'candidate',
       'username', 'accDesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount',
       'favoritecount', 'text', 'tweetsource', 'hashtags'],
      dtype='object')


In [16]:
print(data.columns)

Index(['id;created_at;text;retweet_count;favorite_count', 'candidate',
       'username', 'accDesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount',
       'favoritecount', 'text', 'tweetsource', 'hashtags'],
      dtype='object')


In [17]:
import pandas as pd
import plotly.express as px

# Convert likes column to numeric
data['favoritecount'] = pd.to_numeric(
    data['favoritecount'],
    errors='coerce'
)

# Convert retweetcount also (good practice)
data['retweetcount'] = pd.to_numeric(
    data['retweetcount'],
    errors='coerce'
)

# Likes comparison
likes_comparison = (
    data.groupby('candidate')['favoritecount']
    .sum()
    .reset_index(name='likes')
)

fig = px.bar(
    likes_comparison,
    x='candidate',
    y='likes',
    color='candidate',
    labels={'candidate': 'Candidate', 'likes': 'Total Likes'},
    title='Comparison of Likes'
)

fig.show()


In [18]:
print(data.columns)

Index(['id;created_at;text;retweet_count;favorite_count', 'candidate',
       'username', 'accDesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount',
       'favoritecount', 'text', 'tweetsource', 'hashtags'],
      dtype='object')


In [19]:
import plotly.express as px

top10countries = (
    data.groupby('location')['text']
    .count()
    .sort_values(ascending=False)
    .reset_index(name='tweet_count')
    .head(10)
)

fig = px.bar(
    top10countries,
    x='location',
    y='tweet_count',
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Dark24_r,
    title='Top 10 Location-wise Tweet Counts'
)

fig.show()


In [20]:
print(data.columns)

Index(['id;created_at;text;retweet_count;favorite_count', 'candidate',
       'username', 'accDesc', 'location', 'following', 'followers',
       'totaltweets', 'usercreated', 'tweetcreated', 'retweetcount',
       'favoritecount', 'text', 'tweetsource', 'hashtags'],
      dtype='object')


In [21]:
import plotly.express as px

# Step 1: Top 10 locations
top10locations = (
    data.groupby('location')['text']
    .count()
    .sort_values(ascending=False)
    .reset_index(name='tweet_count')
    .head(10)
)

# Step 2: Tweets per candidate per location
tweet_df = (
    data.groupby(['location', 'candidate'])['text']
    .count()
    .reset_index(name='tweet_count')
)

# Step 3: Filter top 10 locations
tweeters = tweet_df[
    tweet_df['location'].isin(top10locations['location'])
]

# Step 4: Plot
fig = px.bar(
    tweeters,
    x='location',
    y='tweet_count',
    color='candidate',
    labels={
        'location': 'Location',
        'tweet_count': 'Number of Tweets',
        'candidate': 'Candidate'
    },
    title='Tweet Counts for Each Candidate in the Top 10 Locations',
    template='plotly_dark',
    barmode='group'
)

fig.show()


In [22]:
def clean(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', str(text))

    # Convert text to lowercase
    text = text.lower()

    # Replace anything other than alphabets a-z with a space
    text = re.sub('[^a-z]', ' ', text)

    # Split the text into single words
    text = text.split()

    # Initialize WordNetLemmatizer
    lm = WordNetLemmatizer()

    # Lemmatize words and remove stopwords
    text = [lm.lemmatize(word) for word in text if word not in set(
        stopwords.words('english'))]

    # Join the words back into a sentence
    text = ' '.join(word for word in text)

    return text

In [23]:
def getpolarity(text):
    return TextBlob(text).sentiment.polarity

def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getAnalysis(score):
    if score < 0:
        return 'negative'
    elif score == 0:
        return 'neutral'
    else:
        return 'positive'

In [24]:
# Filter Trump tweets (case-sensitive fix)
trump_tweets = data[data['candidate'] == 'Trump']

# Filter US tweets using location
trump_tweets = trump_tweets[
    trump_tweets['location'].str.contains(
        'US|United States|USA', case=False, na=False
    )
]

# Select tweet text column
trump_tweets = trump_tweets[['text']]

print(trump_tweets.head())


Empty DataFrame
Columns: [text]
Index: []


In [25]:
import re

def clean(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)   # URLs remove
    text = re.sub(r'@\w+', '', text)             # mentions remove
    text = re.sub(r'#\w+', '', text)             # hashtags remove
    text = re.sub(r'[^a-z\s]', '', text)         # special chars remove
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [26]:
trump_tweets['cleantext'] = trump_tweets['text'].apply(clean)
print(trump_tweets.head())


Empty DataFrame
Columns: [text, cleantext]
Index: []


In [27]:
trump_tweets['subjectivity'] = trump_tweets['cleantext'].apply(getsubjectivity)

In [28]:
from textblob import TextBlob

# make sure cleantext column exists
trump_tweets = trump_tweets.copy()

trump_tweets['polarity'] = trump_tweets['cleantext'].apply(
    lambda x: TextBlob(x).sentiment.polarity
)


In [29]:
def getAnalysis(score):
    if score > 0:
        return 'Positive'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Negative'


In [30]:
trump_tweets['analysis'] = trump_tweets['polarity'].apply(getAnalysis)
trump_tweets.head()


,text,cleantext,subjectivity,polarity,analysis


In [31]:
print(trump_tweets.shape)
print(trump_tweets['analysis'].value_counts(dropna=False))



(0, 5)
Series([], Name: count, dtype: int64)


In [32]:
import matplotlib.pyplot as plt

plt.style.use('dark_background')

sentiment_counts = trump_tweets['analysis'].value_counts(normalize=True)

if sentiment_counts.empty:
    print("❌ No sentiment data available to plot")
else:
    sentiment_counts = sentiment_counts * 100

    sentiment_counts.reindex(
        ['Positive', 'Neutral', 'Negative']
    ).plot.bar(
        color=['green', 'blue', 'red']
    )

    plt.ylabel("Percentage of Tweets")
    plt.title("Distribution of Sentiments towards Trump")
    plt.show()


❌ No sentiment data available to plot


In [33]:
# cleantext exists?
print(trump_tweets[['text', 'cleantext']].head())

# polarity exists?
print(trump_tweets['polarity'].head())

# analysis exists?
print(trump_tweets['analysis'].head())


Empty DataFrame
Columns: [text, cleantext]
Index: []
Series([], Name: polarity, dtype: object)
Series([], Name: analysis, dtype: object)


In [34]:
import matplotlib.pyplot as plt

plt.style.use('dark_background')

sentiment_counts = trump_tweets['analysis'].value_counts(normalize=True)

if sentiment_counts.empty:
    print("❌ Plot nahi bana: sentiment data empty hai")
else:
    sentiment_counts = sentiment_counts * 100

    sentiment_counts = sentiment_counts.reindex(
        ['Positive', 'Neutral', 'Negative']
    ).fillna(0)

    ax = sentiment_counts.plot.bar(
        color=['green', 'blue', 'red'],
        figsize=(7,5)
    )

    ax.set_ylabel("Percentage of Tweets")
    ax.set_title("Distribution of Sentiments towards Trump")

    plt.tight_layout()
    plt.show()


❌ Plot nahi bana: sentiment data empty hai
